In [1]:
# Template Notebook for Indvidualized Participant Engagement Classifcation NN Model

# ToDo/Goals: https://docs.google.com/document/d/1_dTnbdRoBV0A7xCxVEyLBdckrlsUnMnRFbbRn39tiXk/edit?usp=sharing
# Data: https://drive.google.com/drive/folders/19aJUAlkTMz7PcZE1q4hogFkjVtwYGcMT
# Upload code to help-seeking github: https://github.com/interaction-lab/help_seeking
# Record model results here: https://docs.google.com/spreadsheets/d/16ye54fSSEuAuDL_j56UIeDB-rIIrxq_kPbtyPRQOrVI/edit?usp=sharing

In [2]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics

from keras.layers import Dense, Activation
from keras.models import Sequential

Using TensorFlow backend.


#### TODO: Specifiy Data Path Based on Participant/Your Computer

In [3]:
# TODO: Specify Data Path based on participant/your computer
# Warning: this will probably take some time

file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p8_data_processed.csv'
data = pd.read_csv(file)

/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.head()

,participant,session_num,session_date,timestamp,engagement,activity,skill,difficulty,aptitude,games_total,...,op_p2_pose_y9,skill_NC,skill_OS,skill_EM,diff_1,diff_2,diff_3,diff_4,diff_5,no_game
0,8.0,1.0,2018-06-07,0.000000,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
1,8.0,1.0,2018-06-07,0.033333,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
2,8.0,1.0,2018-06-07,0.066667,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
3,8.0,1.0,2018-06-07,0.100000,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
4,8.0,1.0,2018-06-07,0.133333,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1


In [5]:
# Drop Irrelevant Columns

# transcriptions
data = data.drop(columns=['transcript_spk_0', 'transcript_spk_1', 'transcript_spk_2'])

# raw ros messages
data = data.drop(columns=['ros_PARTICIPANT_STATE', 'ros_ROBOT_STATE'])

# more for counting purposes
data = data.drop(columns=['game_start', 'game_correct', 'game_incorrect', 'mistake_made'])

data = data.drop(columns=['participant', 'session_date'])

# use one-hot encoded difficulty and skill, don't use activity for now
data = data.drop(columns=['difficulty', 'skill', 'activity', 'of_face_id'])

In [6]:
# Open-Face/Open-Pose Columns? 
# Feature Dictionary Documentation here: https://docs.google.com/document/d/1RSygoLwsM1PKEIOoqDOaEo3lRBZBLZJuzbMSY5K_6zA/edit?usp=sharing

only_ofop = []
performance = []
for i in data.columns:
    # open face/open pose columns begin with of_ or op_
    if 'op_' in i or 'of_' in i:
        only_ofop.append(i)
    else:
        performance.append(i)
        
only_ofop.remove('of_timestamp')

In [7]:
# Label Analysis

print(data['engagement'].value_counts())
print()

print(1,data['engagement'].value_counts()[1]/sum(data['engagement'].value_counts()))
print(0,data['engagement'].value_counts()[0]/sum(data['engagement'].value_counts()))

1.0    139463
0.0    120473
Name: engagement, dtype: int64

1 0.5365282223316509
0 0.4634717776683491


### Step 1: Set Up Feature Set
#### (FS4: Performance + Handpicked OF/OP from Caitlyn's Intuition)
Note: keep timestamp and session_num until right before running the model in all feature sets

In [8]:
# Feature Set 4: Performance + Handpicked OF/OP

hand_picked = ['confidence', 'success', 'gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y']
for i,c in enumerate(hand_picked):
    hand_picked[i] = 'of_' + c
hand_picked.append('op_Number of People')

for i in performance:
    hand_picked.append(i)

FS = data[hand_picked]

### Step 2: Train Test Split (for chosen Feature Set)

#### TODO: Specify which train-test split you want

1. Random Shuffle Split: 70-30, 50-50, 30-70
2. Chronological Split (80-20, 70-30, 60-40, 50-50, 40-60, 30-70, 20-80)

In [9]:
# Sort data (don't need for Random Shuffle, but ok to leave it)

data = data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [10]:
# TODO: Uncomment correct train-test split below

y = FS['engagement']
X = FS.drop(columns=['timestamp', 'session_num', 'engagement'])

# Choose one of these 

# 1. Random Shuffle Split 70-30
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Random Shuffle Split 50-50
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# 3. Random Shuffle Split 30-70
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

#############################

# 4. Chronological Split 80-20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 5. Random Shuffle Split 70-30
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# 6. Random Shuffle Split 60-40
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)

# 7. Random Shuffle Split 50-50
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)

# 8. Random Shuffle Split 40-60
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, shuffle=False)

# 9. Random Shuffle Split 30-70
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, shuffle=False)

# 10. Random Shuffle Split 20-80
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, shuffle=False)


### Step 3: Neural Network Model

#### TODO: May need to adjust following if poor accurracy
1. Vary num_epochs, batch size
2. Vary between 2, 3, 4 hidden layers
3. Vary amount of nodes in each layer

In [28]:
num_epochs = 10          # if bad, try 5, 10, 15, 20 
size_batch = 128         # if bad, try 64, 128, 256 

In [29]:
# NN Model for FS4 (all OF/OP)

model = Sequential()

model.add(Dense(units=64, input_dim=len(X_train.columns)))
model.add(Activation('relu'))

model.add(Dense(units=32))
model.add(Activation('relu'))

model.add(Dense(units=1))
model.add(Activation('sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 64)                1920      
_________________________________________________________________
activation_9 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
activation_10 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
_________________________________________________________________
activation_11 (Activation)   (None, 1)                 0         
Total params: 4,033
Trainable params: 4,033
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss='binary_crossentropy',  
              optimizer='Adam',
              metrics=['accuracy'])

### 4. Model Evaluation

In [31]:
model.fit(X_train, y_train, epochs=num_epochs, batch_size=size_batch, verbose=1)

Epoch 1/10
207948/207948 [==============================] - 4s 17us/step - loss: 0.3701 - acc: 0.8321
Epoch 2/10
207948/207948 [==============================] - 3s 14us/step - loss: 0.2788 - acc: 0.8815
Epoch 3/10
207948/207948 [==============================] - 3s 15us/step - loss: 0.2438 - acc: 0.8970
Epoch 4/10
207948/207948 [==============================] - 3s 14us/step - loss: 0.2235 - acc: 0.9042
Epoch 5/10
207948/207948 [==============================] - 3s 14us/step - loss: 0.2092 - acc: 0.9111
Epoch 6/10
207948/207948 [==============================] - 3s 15us/step - loss: 0.1969 - acc: 0.9169
Epoch 7/10
207948/207948 [==============================] - 3s 15us/step - loss: 0.1883 - acc: 0.9209
Epoch 8/10
207948/207948 [==============================] - 3s 15us/step - loss: 0.1800 - acc: 0.9249
Epoch 9/10
207948/207948 [==============================] - 3s 15us/step - loss: 0.1741 - acc: 0.9286
Epoch 10/10
207948/207948 [==============================] - 3s 15us/step - loss: 

In [32]:
loss_and_metrics = model.evaluate(X_test, y_test)
print(loss_and_metrics)

51988/51988 [==============================] - 1s 22us/step
[1.0341490454856355, 0.7591559590674771]


In [33]:
scores = model.predict_proba(X_test)
pred = model.predict_classes(X_test)

#### TODO: RECORD ACCURACY IN MODEL RESULTS SPREADSHEET
Try to get accurracy, AUC above 90%!!!

In [34]:
try:
    print("AUC:",roc_auc_score(y_test, scores)) 
except:
    print("AUC undefinied, only 1 class in test data")
    
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))

AUC undefinied, only 1 class in test data
Accuracy: 0.7591559590674771
[[39467 12521]
 [    0     0]]
              precision    recall  f1-score   support

         0.0       1.00      0.76      0.86     51988
         1.0       0.00      0.00      0.00         0

   micro avg       0.76      0.76      0.76     51988
   macro avg       0.50      0.38      0.43     51988
weighted avg       1.00      0.76      0.86     51988



/anaconda3/envs/caispp/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
